In [4]:
# !pip3 install anndata
!pip3 install boto3

     |████████████████████████████████| 131 kB 10.3 MB/s eta 0:00:01
     |████████████████████████████████| 7.5 MB 10.6 MB/s eta 0:00:01
     |████████████████████████████████| 79 kB 9.2 MB/s  eta 0:00:01
     |████████████████████████████████| 153 kB 19.0 MB/s eta 0:00:01
You should consider upgrading via the '/usr/local/opt/python@3.9/bin/python3.9 -m pip install --upgrade pip' command.


In [11]:
import anndata
import boto3
from os import path

In [12]:
BUCKET_NAME = 'czb-tabula-muris-senis'
BULK_OBJECT_KEY = 'Bulk/anndata/190329_maca_bulk.h5ad'
FACS_OBJECT_KEY = 'Data-objects/tabula-muris-senis-facs-official-raw-obj.h5ad'

LOCAL_BULK_PATH = '/tmp/tabula_bulk_data.h5ad'
LOCAL_FACS_PATH = '/tmp/tabula_facs_data.h5ad'

In [15]:
s3 = boto3.resource('s3')

if not path.exists(LOCAL_BULK_PATH):
    s3.Bucket(BUCKET_NAME).download_file(BULK_OBJECT_KEY, LOCAL_BULK_PATH)

if not path.exists(LOCAL_FACS_PATH):
    s3.Bucket(BUCKET_NAME).download_file(FACS_OBJECT_KEY, LOCAL_FACS_PATH)

In [21]:
bulk_data = anndata.read_h5ad(LOCAL_BULK_PATH)
sc_data = anndata.read_h5ad(LOCAL_FACS_PATH)

In [22]:
# Only look at samples from male mice to minimize confounding
male_ind = sc_data.obs.index[sc_data.obs['sex'] == 'male']
male_data = sc_data[male_ind, :]

male_ind_bulk = bulk_data.obs.index[bulk_data.obs['Sex'] == 'm']
male_data_bulk = bulk_data[male_ind_bulk, :]

In [23]:
# only retain genes which are present in both bulk and sc data sets
joint_indices = sc_data.var.index.intersection(bulk_data.var.index)
male_joint_data = male_data[:,joint_indices]
male_joint_data_bulk = male_data_bulk[:,joint_indices]

In [26]:
# If desired, clear the original data to open up some space
# import gc
# sc_data = None
# bulk_data = None
# gc.collect()

In [25]:
# Get data corresponding to limb muscle samples
muscle_ind = male_joint_data.obs.index[male_joint_data.obs['tissue'] == 'Limb_Muscle']
muscle_subset = male_joint_data[muscle_ind, :]

muscle_ind_bulk = male_joint_data_bulk.obs.index[male_joint_data_bulk.obs['Organ'] == 'Limb_Muscle']
muscle_subset_bulk = male_joint_data_bulk[muscle_ind_bulk, :]

In [27]:
import sys
sys.path.append('../')

In [28]:
MUSCLE_BULK_PATH = '../example_data/muscle_subset_bulk.h5ad'
MUSCLE_FACS_PATH = '../example_data/muscle_subset.h5ad'

In [30]:
if not path.exists(MUSCLE_BULK_PATH):
    muscle_subset_bulk.write(MUSCLE_BULK_PATH)
if not path.exists(MUSCLE_FACS_PATH):
    muscle_subset.write(MUSCLE_FACS_PATH)